In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from catboost import CatBoostRegressor

In [2]:
df = pd.read_csv('../data/refined/imoveis.csv')
df['crawler'].value_counts()

liberdade     1078
saúde         1077
bela_vista    1075
ipiranga      1069
brooklin      1012
broklin       1012
Name: crawler, dtype: int64

In [12]:
df[["area_limpo", "Banheiro", "Quarto", "condominio","preço"]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6323 entries, 0 to 6322
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   area_limpo  6323 non-null   int64  
 1   Banheiro    6323 non-null   int64  
 2   Quarto      6323 non-null   int64  
 3   condominio  6063 non-null   float64
 4   preço       6323 non-null   int64  
dtypes: float64(1), int64(4)
memory usage: 247.1 KB


In [14]:
# Substituir valores nulos por 0 no DataFrame df
df.fillna(0, inplace=True)

In [15]:
df[["area_limpo", "Banheiro", "Quarto", "condominio","preço"]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6323 entries, 0 to 6322
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   area_limpo  6323 non-null   int64  
 1   Banheiro    6323 non-null   int64  
 2   Quarto      6323 non-null   int64  
 3   condominio  6323 non-null   float64
 4   preço       6323 non-null   int64  
dtypes: float64(1), int64(4)
memory usage: 247.1 KB


In [16]:
x = X = df[["area_limpo", "Banheiro", "Quarto", "condominio"]]
y = df["preço"]

In [17]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=42, shuffle=True)

In [18]:
X.shape

(6323, 4)

In [19]:
x_test.shape

(1265, 4)

In [20]:
y_train

4567    1500000
5842    3500000
5113     604000
3333     890000
3382     949000
         ...   
3772     780000
5191    1000000
5226    1200000
5390    1040500
860     1059980
Name: preço, Length: 5058, dtype: int64

In [21]:
#fit_intercept Ele controla se o modelo de regressão linear deve calcular ou não o intercepto (também conhecido como viés ou coeficiente linear)
linear1 = LinearRegression(fit_intercept=True)
linear2 = LinearRegression(fit_intercept=False)

In [22]:
linear1.fit(x_train,y_train)
linear2.fit(x_train,y_train)

LinearRegression(fit_intercept=False)

In [23]:
y_pred_train1 = linear1.predict(x_train)
y_pred_test1 = linear1.predict(x_test)
y_pred_train2 = linear2.predict(x_train)
y_pred_test2 = linear2.predict(x_test)

In [24]:
np.mean(np.abs(y_train - y_pred_train1)/y_train)

0.31305991721100956

In [25]:
np.mean(np.abs(y_train - y_pred_train2)/y_train)

0.3272471980359713

In [26]:
np.mean(np.abs(y_test - y_pred_test1)/y_test)

0.3090014711185113

In [27]:
np.mean(np.abs(y_test - y_pred_test2)/y_test)

0.3207849469867356

In [28]:
r2_score(y_test, y_pred_test1)

0.7424004437105677

In [29]:
r2_score(y_test, y_pred_test2)

0.7407308274784408